In [ ]:
import os
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4-1106-preview"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [2]:
persist_directory = "/Users/dardonacci/Documents/work/ai-assistant/vector_stores"

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_documentation",
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [3]:
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# Build prompt
template = """Use the following pieces of context that are part of the Hummingbot Documentation to answer the user question. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI(model_name=llm_name, temperature=0)
output_parser = StrOutputParser()

chain = prompt | model

In [7]:
query = """Can you explain me how to use the pure market making strategy?
"""

In [8]:
from langchain.schema.runnable import RunnableMap

retriever = vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10)

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [9]:
chain.invoke(
    input={
        "question": query,
    }
)

"Certainly! The pure market making strategy in Hummingbot allows you to run a market making strategy on a single trading pair on spot exchanges. Here's a step-by-step guide on how to use the pure market making strategy:\n\n1. **Choose an Exchange**: First, you need to select a spot exchange that is supported by Hummingbot and set up your API keys within the Hummingbot client.\n\n2. **Select a Trading Pair**: Choose the trading pair you want to make a market in. This will be the pair for which you will be placing buy and sell orders.\n\n3. **Configure Strategy Parameters**: You will need to configure several parameters for the strategy, including:\n   - `bid_spread` and `ask_spread`: The percentage difference from the mid-price where your buy (bid) and sell (ask) limit orders will be placed.\n   - `order_refresh_time`: The frequency in seconds at which your orders will be canceled and new orders will be placed with updated prices.\n   - `order_amount`: The size of each order in terms of

In [12]:
class Strategy(BaseModel):
    """Strategy created by the LLM for the user."""
    code: str = Field(description="Provide the python code for the strategy.")
    summary: str = Field(description="Provide a concise summary of what the strategy is doing.")

In [13]:

strategy_extraction_function = [
    convert_pydantic_to_openai_function(Strategy)
]
tagging_model = model.bind(
    functions=strategy_extraction_function,
    function_call={"name":"Strategy"}
)

In [14]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | StrOutputParser() | tagging_model | JsonOutputFunctionsParser()

In [15]:
result = tagging_chain.invoke(
    input={
        "question": query,
    }
)

In [16]:
print(result["summary"])

The provided Python script is a template for a Directional Strategy using Hummingbot with the MACDBB controller. It defines a class `MACDBBDirectionalStrategy` that extends `ScriptStrategyBase` and includes configuration for the trading pair, exchange, and order amount. The strategy uses the MACDBB controller with initial values for the MACD and Bollinger Bands indicators. The `on_tick` method contains the main trading logic, which checks the MACDBB signal to decide on placing buy or sell orders. Placeholder methods for determining buy/sell prices and order placement need to be implemented. The script requires further development and optimization based on historical data and market conditions before actual trading.


In [17]:
print(result["code"])

from decimal import Decimal
from hummingbot.connector.connector_base import ConnectorBase
from hummingbot.core.data_type.common import OrderType
from hummingbot.smart_components.controllers.macd_bb_v1 import MACDBBV1, MACDBBV1Config
from hummingbot.strategy.script_strategy_base import ScriptStrategyBase

class MACDBBDirectionalStrategy(ScriptStrategyBase):
    # Configuration
    trading_pair = "BTC-USDT"  # Example trading pair
    exchange = "binance"  # Example exchange
    order_amount = Decimal("0.01")  # Example order amount in base currency

    # MACDBB Configuration
    macd_bb_config = MACDBBV1Config(
        fast_ema_period=12,  # Fast EMA period for MACD
        slow_ema_period=26,  # Slow EMA period for MACD
        signal_ema_period=9,  # Signal EMA period for MACD
        bollinger_band_period=20,  # Bollinger Bands period
        bollinger_band_std_dev=2  # Bollinger Bands standard deviation
    )
    macd_bb_controller = None

    def __init__(self, connectors: Dict[st